# 데이터 인코딩
<br> 머신러닝을 위한 대표적 인코딩 방식으로 레이블 인코딩과 원핫 인코딩 존재

## 레이블 인코딩

In [19]:
from sklearn.preprocessing import LabelEncoder

items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','선풍기','믹서','믹서']

# LabelEncoder를 객체로 생성한 후, fit()과 transform()으로 레이블 인코딩 실행

encoder = LabelEncoder()
labels = encoder.fit_transform(items)
labels

array([0, 1, 4, 5, 3, 3, 2, 2])

In [2]:
encoder.classes_

array(['TV', '냉장고', '믹서', '선풍기', '전자레인지', '컴퓨터'], dtype='<U5')

In [3]:
encoder.inverse_transform([4,5,2,0,1,1,2,3])

array(['전자레인지', '컴퓨터', '믹서', 'TV', '냉장고', '냉장고', '믹서', '선풍기'], dtype='<U5')

단, 0, 1 이상의 값이 존재하므로 특정 알고리즘에서 가중치가 높게 계산될 가능성 있음. <br>
특히 선형회귀와 같은 알고리즘. 원학 인코딩은 이런 레이블 인코딩의 문제를 해결함

## 원 핫 인코딩

In [12]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

encoder = OneHotEncoder()

items_ar = np.array(items).reshape(-1,1) # 피쳐를 1개로 맞춰줘야 함
oh_labels = encoder.fit_transform(items_ar)

<8x6 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [16]:
oh_labels.toarray()

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [17]:
oh_labels.shape

(8, 6)

In [20]:
import pandas as pd

df = pd.DataFrame({'item':items})
pd.get_dummies(df)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


# 피쳐스케일링과 정규화

## 표준정규화 - StandardScaler
<br> 개별 피처를 평균 0, 분산이 1인 값으로 변환 (가우시안 정규분포를 가지도록...)
<br> SVM, 선형회귀, 로지스틱 회귀는 가우시안 분포 가정이기에 성능향상에 도움

In [23]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns= iris.feature_names)
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
iris_std = scaler.fit_transform(iris_df)

iris_df_std = pd.DataFrame(iris_std, columns= ['sepal length','sepal width','petal length','petal width'])
iris_df_std.describe().round(2)

,sepal length,sepal width,petal length,petal width
count,150.00,150.00,150.00,150.00
mean,-0.00,-0.00,-0.00,-0.00
std,1.00,1.00,1.00,1.00
min,-1.87,-2.43,-1.57,-1.45
25%,-0.90,-0.59,-1.23,-1.18
50%,-0.05,-0.13,0.34,0.13
75%,0.67,0.56,0.76,0.79
max,2.49,3.09,1.79,1.71


## -1 또는 0과 1 범위의 값으로 변환 - MinMaxScaler
<br> 데이터 분포가 가우시안 분포가 아닐 경우 적용

In [31]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
iris_minmax = scaler.fit_transform(iris_df)

iris_df_minmax = pd.DataFrame(iris_minmax, columns= ['sepal length','sepal width','petal length','petal width'])
iris_df_minmax.describe().round(2)


,sepal length,sepal width,petal length,petal width
count,150.00,150.00,150.00,150.00
mean,0.43,0.44,0.47,0.46
std,0.23,0.18,0.30,0.32
min,0.00,0.00,0.00,0.00
25%,0.22,0.33,0.10,0.08
50%,0.42,0.42,0.57,0.50
75%,0.58,0.54,0.69,0.71
max,1.00,1.00,1.00,1.00


## 스케일링 시 유의점

1. 가능하다면 전체 데이터의 스케일링 변환을 적용한 뒤 학습/테스트 데이터 분리
2. 1이 여의치 않다면 테스트데이터 변환시에는 fit()이나 fit_transform()을 적용하지 않고<br> 학습 데이터로 이미 fit()된 Scaler객체를 이용하여 tranform()으로 변환